In [1]:
from kiteconnect import KiteConnect
import os
import datetime as dt
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from time import sleep 
import statsmodels.api as sm
from pyotp import TOTP
from datetime import datetime
from urllib.parse import urlparse,parse_qs
import matplotlib.pyplot as plt
import mplfinance as mpf
import plotly.graph_objects as go
import seaborn as sns

# Automating the kite connect login via selenium

In [2]:
def get_curr_path(folder_name):
    curr_dir = os.getcwd()
    curr_path = os.path.join(curr_dir,folder_name)
    return curr_path

def get_credentials(curr_path,file_name):
    file_dir = os.path.join(curr_path,file_name)
    file = open(file_dir,'r').read().split()
    api_key = file[0]
    api_secret = file[1]
    user_name = file[2]
    pwd = file[3]
    totp_key = file[-1]
    return api_key,api_secret,user_name,pwd,totp_key

def auto_login(api_key,user_name,pwd,totp_key):
    kite = KiteConnect(api_key=api_key)
    service = Service(ChromeDriverManager().install())
    service.start()
    options = Options()
    options.to_capabilities()
    driver = webdriver.Remote(
        command_executor=service.service_url,
        options=options)
    driver.get(kite.login_url())
    driver.implicitly_wait(5)
    username = driver.find_element(By.XPATH, "//input[@type='text']")
    username.send_keys(user_name)
    password = driver.find_element(By.XPATH, "//input[@type='password']")
    password.send_keys(pwd)
    driver.find_element(By.XPATH, "//button[@type='submit']").click()
    sleep(1)
    totp = driver.find_element(By.XPATH,"//input[@type='number']")
    totp_token = TOTP(totp_key)
    token = totp_token.now()
    totp.send_keys(token)
    driver.find_element(By.XPATH,"//button[@type = 'submit']").click()
    sleep(1)
    current_url = driver.current_url
    parsed_url = urlparse(current_url)
    query_params = parse_qs(parsed_url.query)
    request_token = query_params.get('request_token',[None])[0]
    with open('request_token.txt', 'w') as f:
        f.write(request_token)
    request_token = open('request_token.txt','r').read()
    driver.quit()
    return request_token
    

def generate_access_token(request_token,api_key,api_secret):
    request_token = open('request_token.txt','r').read()
    kite = KiteConnect(api_key=api_key)
    data = kite.generate_session(request_token=request_token,api_secret=api_secret)
    data
    with open('access_token.txt','w') as f:
        f.write(data['access_token'])
    access_token = open('access_token.txt','r').read()
    return access_token

In [3]:
folder_name = 'api_keys'
curr_dir = get_curr_path(folder_name)

file_name = 'credentials.txt'
api_key,api_secret,user_name,pwd,totp_key = get_credentials(curr_dir,file_name)

request_token = auto_login(api_key,user_name,pwd,totp_key)

access_token = generate_access_token(request_token,api_key,api_secret)

# Setting up the access token to execute buy,sell orders

In [4]:
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token=access_token)

In [5]:
instrument_dump = kite.instruments('NSE')
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv('NSE_instruments.csv',index=False)

# Retrieve historical data 

In [6]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1
    
def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data

def fetchOHLCExtended(ticker, inception_date, interval):
    """Extracts historical data and outputs in the form of a DataFrame.
       inception_date string format - dd-mm-yyyy"""
    instrument = instrumentLookup(instrument_df, ticker)
    from_date = dt.datetime.strptime(inception_date, '%d-%m-%Y')
    data = pd.DataFrame()  # Start with an empty DataFrame
    while True:
        if from_date.date() >= (dt.date.today() - dt.timedelta(100)):
            new_data = pd.DataFrame(kite.historical_data(instrument, from_date, dt.date.today(), interval))
            if not new_data.empty:
                if data.empty:
                    data = new_data
                else:
                    data = pd.concat([data, new_data], ignore_index=True)
            break
        else:
            to_date = from_date + dt.timedelta(100)
            new_data = pd.DataFrame(kite.historical_data(instrument, from_date, to_date, interval))
            if not new_data.empty:
                if data.empty:
                    data = new_data
                else:
                    data = pd.concat([data, new_data], ignore_index=True)
            from_date = to_date
    
    if not data.empty:
        data.set_index("date", inplace=True)
    return data

# Setting up the super trend strategy

In [7]:
def compute_super_trend(df, n, multiplier):
    df = df.copy()
    df['hl2'] = (df['high'] + df['low']) / 2
    df['tr'] = np.maximum.reduce([df['high'] - df['low'],
                                  abs(df['high'] - df['close'].shift(1)),
                                  abs(df['low'] - df['close'].shift(1))])
    alpha = 1 / n
    
    df.loc[df.index[n-1], 'atr'] = df['tr'][:n].mean()
    for i in range(n, len(df)):
        df.loc[df.index[i], 'atr'] = alpha * df.loc[df.index[i], 'tr'] + (1 - alpha) * df.loc[df.index[i-1], 'atr']

    df['basic_ub'] = df['hl2'] + (multiplier * df['atr'])
    df['basic_lb'] = df['hl2'] - (multiplier * df['atr'])
    df.loc[df.index[n-1], 'ub'] = df.loc[df.index[n-1], 'basic_ub']
    df.loc[df.index[n-1], 'lb'] = df.loc[df.index[n-1], 'basic_lb']

    for i in range(n, len(df)):
        if ((df.loc[df.index[i], 'basic_ub'] < df.loc[df.index[i-1], 'ub']) or
            (df.loc[df.index[i-1], 'close'] > df.loc[df.index[i-1], 'ub'])):
            df.loc[df.index[i], 'ub'] = df.loc[df.index[i], 'basic_ub']
        else:
            df.loc[df.index[i], 'ub'] = df.loc[df.index[i-1], 'ub']
        
        if ((df.loc[df.index[i], 'basic_lb'] > df.loc[df.index[i-1], 'lb']) or
            (df.loc[df.index[i-1], 'close'] < df.loc[df.index[i-1], 'lb'])):
            df.loc[df.index[i], 'lb'] = df.loc[df.index[i], 'basic_lb']
        else:
            df.loc[df.index[i], 'lb'] = df.loc[df.index[i-1], 'lb']

    isUpTrend, isDownTrend = 'up', 'down'
    init_trend_dir = isDownTrend
    df['trend_direction'] = init_trend_dir
    df['super_trend'] = pd.Series(dtype=float)
    for i in range(len(df)):
        if i < n:
            df.loc[df.index[i], 'trend_direction'] = init_trend_dir
        else:
            prev_super_trend = df.loc[df.index[i-1], 'super_trend'] if i > n else None
            prev_upper_band = df.loc[df.index[i-1], 'ub']
            prev_lower_band = df.loc[df.index[i-1], 'lb']
            
            if prev_super_trend == prev_upper_band:
                if df.loc[df.index[i], 'close'] > df.loc[df.index[i], 'ub']:
                    df.loc[df.index[i], 'trend_direction'] = isUpTrend
                else:
                    df.loc[df.index[i], 'trend_direction'] = isDownTrend
            else:
                if df.loc[df.index[i], 'close'] < df.loc[df.index[i], 'lb']:
                    df.loc[df.index[i], 'trend_direction'] = isDownTrend
                else:
                    df.loc[df.index[i], 'trend_direction'] = isUpTrend
            
            if df.loc[df.index[i], 'trend_direction'] == isUpTrend:
                df.loc[df.index[i], 'super_trend'] = df.loc[df.index[i], 'lb']
            else:
                df.loc[df.index[i], 'super_trend'] = df.loc[df.index[i], 'ub']
            
            if i >= len(df) - 5: 
                print(f"Index: {i}, Close: {df.loc[df.index[i], 'close']}, "
                      f"UB: {df.loc[df.index[i], 'ub']}, LB: {df.loc[df.index[i], 'lb']}, "
                      f"SuperTrend: {df.loc[df.index[i], 'super_trend']}, "
                      f"TrendDirection: {df.loc[df.index[i], 'trend_direction']}")

    return df['super_trend']


In [8]:
infy_df = fetchOHLC('INFY','5minute',5)

In [9]:
infy_df.to_csv('infy.csv')

In [10]:
infy_df.head()

,open,high,low,close,volume
date,,,,,
2024-06-10 09:15:00+05:30,1525.30,1529.80,1516.00,1516.95,439043
2024-06-10 09:20:00+05:30,1516.75,1517.40,1514.20,1516.40,279808
2024-06-10 09:25:00+05:30,1516.80,1516.95,1514.15,1514.15,111061
2024-06-10 09:30:00+05:30,1514.20,1514.50,1511.10,1511.40,242386
2024-06-10 09:35:00+05:30,1511.75,1511.75,1507.00,1507.10,200717


In [11]:
copy_df = infy_df.copy()

In [12]:
st1 = compute_super_trend(copy_df,7,3)
st2 = compute_super_trend(copy_df,10,3)
st3 = compute_super_trend(copy_df,11,2)

Index: 282, Close: 1491.25, UB: 1494.559872034272, LB: 1487.2549919748205, SuperTrend: 1494.559872034272, TrendDirection: down
Index: 283, Close: 1492.5, UB: 1494.559872034272, LB: 1487.2549919748205, SuperTrend: 1494.559872034272, TrendDirection: down
Index: 284, Close: 1494.1, UB: 1494.559872034272, LB: 1487.2549919748205, SuperTrend: 1494.559872034272, TrendDirection: down
Index: 285, Close: 1493.95, UB: 1494.559872034272, LB: 1487.6138579200037, SuperTrend: 1494.559872034272, TrendDirection: down
Index: 286, Close: 1493.7, UB: 1494.559872034272, LB: 1488.5225925028606, SuperTrend: 1494.559872034272, TrendDirection: down
Index: 282, Close: 1491.25, UB: 1494.5228971566278, LB: 1489.0791043185068, SuperTrend: 1494.5228971566278, TrendDirection: down
Index: 283, Close: 1492.5, UB: 1494.5228971566278, LB: 1489.0791043185068, SuperTrend: 1494.5228971566278, TrendDirection: down
Index: 284, Close: 1494.1, UB: 1494.5228971566278, LB: 1489.0791043185068, SuperTrend: 1494.5228971566278, Tren

In [13]:
copy_df['st1'] = st1
copy_df['st2'] = st2
copy_df['st3'] = st3

In [14]:
copy_df

,open,high,low,close,volume,st1,st2,st3
date,,,,,,,,
2024-06-10 09:15:00+05:30,1525.30,1529.80,1516.00,1516.95,439043,NaN,NaN,NaN
2024-06-10 09:20:00+05:30,1516.75,1517.40,1514.20,1516.40,279808,NaN,NaN,NaN
2024-06-10 09:25:00+05:30,1516.80,1516.95,1514.15,1514.15,111061,NaN,NaN,NaN
2024-06-10 09:30:00+05:30,1514.20,1514.50,1511.10,1511.40,242386,NaN,NaN,NaN
2024-06-10 09:35:00+05:30,1511.75,1511.75,1507.00,1507.10,200717,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2024-06-13 14:00:00+05:30,1491.80,1493.55,1491.15,1491.25,53222,1494.559872,1494.522897,1493.021008
2024-06-13 14:05:00+05:30,1491.10,1493.40,1491.00,1492.50,57523,1494.559872,1494.522897,1493.021008
2024-06-13 14:10:00+05:30,1492.50,1494.30,1491.45,1494.10,52875,1494.559872,1494.522897,1489.269729


In [15]:
copy_df.to_csv('three_st.csv')

# Setting up the signals 

In [16]:
def st_dir_refresh(ohlc,ticker,st_dir):
    # check for trend reversal
    if (ohlc.loc[ohlc.index[-1],'st1'] > ohlc.loc[ohlc.index[-1],'close']) and (ohlc.loc[ohlc.index[-2],'st1'] < ohlc.loc[ohlc.index[-2],'close']):
        st_dir[ticker][0] = 'red'
    if (ohlc.loc[ohlc.index[-1],'st2'] > ohlc.loc[ohlc.index[-1],'close']) and (ohlc.loc[ohlc.index[-2],'st2'] < ohlc.loc[ohlc.index[-2],'close']):
        st_dir[ticker][1] = 'red'
    if (ohlc.loc[ohlc.index[-1],'st3'] > ohlc.loc[ohlc.index[-1],'close']) and (ohlc.loc[ohlc.index[-2],'st3'] < ohlc.loc[ohlc.index[-2],'close']):
        st_dir[ticker][2] = 'red'
    if (ohlc.loc[ohlc.index[-1],'st1'] < ohlc.loc[ohlc.index[-1],'close']) and (ohlc.loc[ohlc.index[-2],'st1'] > ohlc.loc[ohlc.index[-2],'close']):
        st_dir[ticker][0] = 'green'
    if (ohlc.loc[ohlc.index[-1],'st2'] < ohlc.loc[ohlc.index[-1],'close']) and (ohlc.loc[ohlc.index[-2],'st2'] > ohlc.loc[ohlc.index[-2],'close']):
        st_dir[ticker][1] = 'green'
    if (ohlc.loc[ohlc.index[-1],'st3'] < ohlc.loc[ohlc.index[-1],'close']) and (ohlc.loc[ohlc.index[-2],'st3'] > ohlc.loc[ohlc.index[-2],'close']):
        st_dir[ticker][2] = 'green'
    return st_dir 

In [17]:
st_df = pd.read_csv('three_st.csv')
st_df.set_index('date',inplace=True)
st_df

,open,high,low,close,volume,st1,st2,st3
date,,,,,,,,
2024-06-10 09:15:00+05:30,1525.30,1529.80,1516.00,1516.95,439043,NaN,NaN,NaN
2024-06-10 09:20:00+05:30,1516.75,1517.40,1514.20,1516.40,279808,NaN,NaN,NaN
2024-06-10 09:25:00+05:30,1516.80,1516.95,1514.15,1514.15,111061,NaN,NaN,NaN
2024-06-10 09:30:00+05:30,1514.20,1514.50,1511.10,1511.40,242386,NaN,NaN,NaN
2024-06-10 09:35:00+05:30,1511.75,1511.75,1507.00,1507.10,200717,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2024-06-13 14:00:00+05:30,1491.80,1493.55,1491.15,1491.25,53222,1494.559872,1494.522897,1493.021008
2024-06-13 14:05:00+05:30,1491.10,1493.40,1491.00,1492.50,57523,1494.559872,1494.522897,1493.021008
2024-06-13 14:10:00+05:30,1492.50,1494.30,1491.45,1494.10,52875,1494.559872,1494.522897,1489.269729


In [18]:
st_dir = {}
ticker = 'INFY'
st_dir[ticker] = ['None', 'None', 'None']
# Edge case what if the 2 consecutive days have the same values it would get initialised to None,none,None 
st_dir = st_dir_refresh(st_df, ticker, st_dir)
print(f"Updated st_dir: {st_dir}")

Updated st_dir: {'INFY': ['None', 'None', 'None']}


In [22]:
def sl_price(ohlc):
    """function to calculate stop loss based on supertrends"""
    st = ohlc.iloc[-1,[-3,-2,-1]]
    if st.min() > ohlc.loc[ohlc.index[-1],'close']:
        sl = (0.6*st.sort_values(ascending = True)[0]) + (0.4*st.sort_values(ascending = True)[1])
    elif st.max() < ohlc.loc[ohlc.index[-1],'close']:
        sl = (0.6*st.sort_values(ascending = False)[0]) + (0.4*st.sort_values(ascending = False)[1])
    else:
        sl = st.mean()
    return round(sl,1)

In [26]:
sl = sl_price(copy_df)
print(f"Stop loss: Rs. {sl}")

Stop loss: Rs 1493.1


In [ ]:
def placeSLOrder(symbol,buy_sell,quantity,sl_price):    
    # Place an intraday stop loss order on NSE
    if buy_sell == "buy":
        t_type=kite.TRANSACTION_TYPE_BUY
        t_type_sl=kite.TRANSACTION_TYPE_SELL
    elif buy_sell == "sell":
        t_type=kite.TRANSACTION_TYPE_SELL
        t_type_sl=kite.TRANSACTION_TYPE_BUY
    kite.place_order(tradingsymbol=symbol,
                    exchange=kite.EXCHANGE_NSE,
                    transaction_type=t_type,
                    quantity=quantity,
                    order_type=kite.ORDER_TYPE_MARKET,
                    product=kite.PRODUCT_MIS,
                    variety=kite.VARIETY_REGULAR)
    kite.place_order(tradingsymbol=symbol,
                    exchange=kite.EXCHANGE_NSE,
                    transaction_type=t_type_sl,
                    quantity=quantity,
                    order_type=kite.ORDER_TYPE_SL,
                    price=sl_price,
                    trigger_price = sl_price,
                    product=kite.PRODUCT_MIS,
                    variety=kite.VARIETY_REGULAR)    

In [ ]:
def ModifyOrder(order_id,price):    
    # Modify order given order id
    kite.modify_order(order_id=order_id,
                    price=price,
                    trigger_price=price,
                    order_type=kite.ORDER_TYPE_SL,
                    variety=kite.VARIETY_REGULAR)  